In [1]:
from keras.models import load_model
from time import sleep
from tensorflow.keras.utils import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
import pyvirtualcam
from PIL import Image
import tkinter as tk
from threading import Thread
import socket

In [2]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
classifier =load_model('modelrestrans.h5')
emotion_labels = ["Angry","Happy", "Surprise", "Neutral", "Bored"]

In [3]:
happy=cv2.imread('D:/ACS/mihnea/happy.png', cv2.IMREAD_UNCHANGED)
angry=cv2.imread('D:/ACS/mihnea/angry.png', cv2.IMREAD_UNCHANGED)
neutral=cv2.imread('D:/ACS/mihnea/neutral.png', cv2.IMREAD_UNCHANGED)
surprise=cv2.imread('D:/ACS/mihnea/surprise.png', cv2.IMREAD_UNCHANGED)
bored=cv2.imread('D:/ACS/mihnea/bored.png', cv2.IMREAD_UNCHANGED)

In [4]:
def run_camera():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((socket.gethostname(), 1900))
    s.connect((socket.gethostname(), 1234))
    
    client_id = 'client1'
    s.send(bytes(client_id, 'utf-8'))
    
    cap = cv2.VideoCapture(0)
    while running:
        _, frame = cap.read()
        labels = []
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray)

        if isinstance(faces, tuple):
            s.sendall(b'Bored')
        else:
            for (x, y, w, h) in faces:
                roi_gray = gray[y:y+h, x:x+w]
                roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

                if np.sum([roi_gray]) != 0:
                    roi = roi_gray.astype('float') / 255.0
                    roi = np.expand_dims(roi, axis=0)

                    prediction = classifier.predict(roi)[0]
                    label = emotion_labels[prediction.argmax()]

                    if label == 'Happy':
                        s.sendall(b'Happy')
                    elif label == 'Angry':
                        s.sendall(b'Angry')
                    elif label == 'Bored':
                        s.sendall(b'Bored')
                    elif label == 'Neutral':
                        s.sendall(b'Neutral')
                    elif label == 'Surprise':
                        s.sendall(b'Surprise')

    cap.release()
    s.close()
        
def on_button2_click():
    global running
    running = True
    thread = Thread(target=run_camera)
    thread.start()
    
def stop_camera():
    global running
    running = False

In [20]:
root = tk.Tk()
root.geometry("150x250")
avatar_label = tk.Label(root, text="Avatar")
avatar_label.pack(pady=50)

button2 = tk.Button(root, text="Pornește camera", command=on_button2_click)
button2.pack(pady=5)

button = tk.Button(root, text="Oprește camera", command=stop_camera)
button.pack(pady=5)

exit_button = tk.Button(root, text="Ieșire", command=root.destroy)
exit_button.pack(pady=5)

root.mainloop()

1/1 [==============================] - 0s 43ms/step
